In [11]:
import requests
import json
import pandas as pd
import plotly.express as px
from datetime import datetime

In [12]:
# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",  # Replace with actual sensor IDs
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',            # or 'csv' if you prefer CSV format
    'from-date': '2024-05-18T19:00:00Z',
    'to-date': datetime.isoformat() + 'Z',  # Current date and time in UTC
    'download': 'true',          # This forces a download if format is 'csv'
    'outliers': 'mark',          # Optional: 'mark' or 'replace' if outlier detection is needed
    'outlier-window': 15,        # Optional: default is 15, can be 1-50
    'delimiter': 'comma'         # Optional: 'comma' or 'semicolon' for CSV
}

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    # Construct the endpoint URL
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    
    # Send the GET request to the API
    response = requests.get(endpoint, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert response data to JSON
        data = response.json()
        # Add sensor ID to each measurement
        for measurement in data:
            measurement['Id'] = sensorId
        # Append data to the list
        all_data.extend(data)
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')


In [13]:
# Convert the list of data to a DataFrame
df = pd.DataFrame(all_data)

# Convert the 'createdAt' column to datetime
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Pivot the DataFrame so that each sensor's data is in its own column
sense_df = df.pivot_table(index='createdAt', columns='Id', values='value', aggfunc='first')
sense_df.reset_index(inplace=True)

# Rename columns to more descriptive names
sense_df.rename(columns={'Id': 'Index',
                         'createdAt': 'Messzeitpunkt',
                         '6645db6eeb5aad0007a6e4b7': 'Temperatur',
                         '6645db6eeb5aad0007a6e4b8': 'rel. Luftfeuchte',
                         '6645db6eeb5aad0007a6e4b9': 'Luftdruck',
                         '6645db6eeb5aad0007a6e4ba': 'UV-Intensität'}, inplace=True)

In [14]:
sense_df

Id,Messzeitpunkt,Temperatur,rel. Luftfeuchte,Luftdruck,UV-Intensität
0,2024-05-18 19:00:21.702000+00:00,15.82,77.48,998.12,5.62
1,2024-05-18 19:01:21.937000+00:00,15.69,77.58,998.12,5.62
2,2024-05-18 19:02:22.121000+00:00,15.57,78.42,998.17,0.00
3,2024-05-18 19:03:22.220000+00:00,15.57,78.42,998.19,0.00
4,2024-05-18 19:04:22.256000+00:00,15.56,78.93,998.15,0.00
...,...,...,...,...,...
3723,2024-05-21 09:10:42.584000+00:00,16.00,89.48,994.16,90.00
3724,2024-05-21 09:11:42.491000+00:00,16.15,89.14,994.13,106.88
3725,2024-05-21 09:12:42.685000+00:00,16.28,88.02,994.13,106.88
3726,2024-05-21 09:13:42.770000+00:00,16.39,87.58,994.12,118.12


In [15]:
sense_df.dtypes

Id
Messzeitpunkt       datetime64[ns, UTC]
Temperatur                       object
rel. Luftfeuchte                 object
Luftdruck                        object
UV-Intensität                    object
dtype: object

In [16]:
# Optionally save the DataFrame to a CSV file
sense_df.to_csv('sensor_data.csv', index=False)